In [3]:
import shioaji as sj
import pysimulation

api = sj.Shioaji(simulation=True)
accounts = api.login(
    api_key="",     # 請修改此處
    secret_key=""   # 請修改此處
)

2023-03-12 21:12:48.821 | WARNING  | importlib._bootstrap:_call_with_frames_removed:219 - Optional: pip install shioaji[speed] for better performance.


Response Code: 0 | Event Code: 0 | Info: host '203.66.91.161:80', hostname '203.66.91.161:80' IP 203.66.91.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total connection attempt 1 of 1) | Event: Session up


In [7]:
import datetime
import pandas as pd
import os

days = 365

# last_days = datetime.datetime.now() - datetime.timedelta(days=days)
last_days = datetime.datetime(2022, 3, 5)


ticks = api.ticks(
    contract=api.Contracts.Futures.TXF.TXFR1, 
    date=last_days.strftime("%Y-%m-%d"),
    query_type=sj.constant.TicksQueryType.RangeTime,
    time_start="08:45:00",
    time_end="13:45:01"
)
print(last_days.strftime("%Y-%m-%d"))
df = pd.DataFrame({**ticks})
df.ts = pd.to_datetime(df.ts)
df = df.set_index('ts')
df.index.name = None
df_tick = df

for i in range(days-1):
    date = last_days + datetime.timedelta(days=i+1)
    print(date.strftime("%Y-%m-%d"))
    ticks = api.ticks(
        contract=api.Contracts.Futures.TXF.TXFR1, 
        date=date.strftime("%Y-%m-%d")
    )
    if ticks['ts'] != []:
        df = pd.DataFrame({**ticks})
        df.ts = pd.to_datetime(df.ts)
        df = df.set_index('ts')
        df.index.name = None
        df_tick = pd.concat([df_tick, df], axis=0)

df_tick.to_csv('TXF_tick.csv')


df = pd.read_csv(f"TXF_tick.csv", index_col=0)
df.index = pd.to_datetime(df.index)
resample_ohlc = df['close'].resample('1m').ohlc()
resample_volume = df['volume'].resample('1m').sum()
df_resample = pd.concat([resample_ohlc, resample_volume], axis=1)
df_resample = df_resample.rename(columns={"open": "Open", "high": "High", "low": "Low", "close": "Close", "volume": "Volume"})
df_resample.dropna(axis=0, inplace=True)
# df_resample.head()

2022-03-05
2022-03-06
2022-03-07
2022-03-08
2022-03-09
2022-03-10
2022-03-11
2022-03-12
2022-03-13
2022-03-14
2022-03-15
2022-03-16
2022-03-17
2022-03-18
2022-03-19
2022-03-20
2022-03-21
2022-03-22
2022-03-23
2022-03-24
2022-03-25
2022-03-26
2022-03-27
2022-03-28
2022-03-29
2022-03-30
2022-03-31
2022-04-01
2022-04-02
2022-04-03
2022-04-04
2022-04-05
2022-04-06
2022-04-07
2022-04-08
2022-04-09
2022-04-10
2022-04-11
2022-04-12
2022-04-13
2022-04-14
2022-04-15
2022-04-16
2022-04-17
2022-04-18
2022-04-19
2022-04-20
2022-04-21
2022-04-22
2022-04-23
2022-04-24
2022-04-25
2022-04-26
2022-04-27
2022-04-28
2022-04-29
2022-04-30
2022-05-01
2022-05-02
2022-05-03
2022-05-04
2022-05-05
2022-05-06
2022-05-07
2022-05-08
2022-05-09
2022-05-10
2022-05-11
2022-05-12
2022-05-13
2022-05-14
2022-05-15
2022-05-16
2022-05-17
2022-05-18
2022-05-19
2022-05-20
2022-05-21
2022-05-22
2022-05-23
2022-05-24
2022-05-25
2022-05-26
2022-05-27
2022-05-28
2022-05-29
2022-05-30
2022-05-31
2022-06-01
2022-06-02
2022-06-03

In [8]:
from backtesting.lib import OHLCV_AGG

days = 365

# last_days = datetime.datetime.now() - datetime.timedelta(days=days)
last_days = datetime.datetime(2022, 3, 6)

kbars = api.kbars(
    contract=api.Contracts.Futures.TXF.TXFR1, 
    start=last_days.strftime("%Y-%m-%d"), 
    end=last_days.strftime("%Y-%m-%d"),
)

df = pd.DataFrame({**kbars})
df.ts = pd.to_datetime(df.ts)
df = df.set_index('ts')
df.index.name = None
df_1min = df

for i in range(days-1):
    date = last_days + datetime.timedelta(days=i+1)
    print(date.strftime("%Y-%m-%d"))
    kbars = api.kbars(
        contract=api.Contracts.Futures.TXF.TXFR1, 
        start=date.strftime("%Y-%m-%d"), 
        end=date.strftime("%Y-%m-%d"),
    )
    if kbars['ts'] != []:
        df = pd.DataFrame({**kbars})
        df.ts = pd.to_datetime(df.ts)
        df = df.set_index('ts')
        df.index.name = None
        df_1min = pd.concat([df_1min, df], axis=0)

df_1min = df_1min.between_time('08:45:00', '13:45:01')
df_1min = df_1min[['Open', 'High', 'Low', 'Close', 'Volume']]
df_1min.to_csv('TXF_1min.csv')

df = pd.read_csv(f"TXF_1min.csv", index_col=0)
df.index = pd.to_datetime(df.index)
df = df.resample('5T', label='right').agg(OHLCV_AGG).dropna()


'''
df = pd.read_csv(f"TXF_1min.csv", index_col=0)
df.index = pd.to_datetime(df.index)
resample_open = df['Open'].resample('5min').first()
resample_high = df['High'].resample('5min').max()
resample_low = df['Low'].resample('5min').min()
resample_close = df['Close'].resample('5min').last()
resample_volume = df['Volume'].resample('5Min').sum()
df_resample = pd.concat([resample_open, resample_high, resample_low, resample_close, resample_volume], axis=1)
df_resample.dropna(axis=0, inplace=True)
# df_resample.head()
'''

2022-03-07
2022-03-08
2022-03-09
2022-03-10
2022-03-11
2022-03-12
2022-03-13
2022-03-14
2022-03-15
2022-03-16
2022-03-17
2022-03-18
2022-03-19
2022-03-20
2022-03-21
2022-03-22
2022-03-23
2022-03-24
2022-03-25
2022-03-26
2022-03-27
2022-03-28
2022-03-29
2022-03-30
2022-03-31
2022-04-01
2022-04-02
2022-04-03
2022-04-04
2022-04-05
2022-04-06
2022-04-07
2022-04-08
2022-04-09
2022-04-10
2022-04-11
2022-04-12
2022-04-13
2022-04-14
2022-04-15
2022-04-16
2022-04-17
2022-04-18
2022-04-19
2022-04-20
2022-04-21
2022-04-22
2022-04-23
2022-04-24
2022-04-25
2022-04-26
2022-04-27
2022-04-28
2022-04-29
2022-04-30
2022-05-01
2022-05-02
2022-05-03
2022-05-04
2022-05-05
2022-05-06
2022-05-07
2022-05-08
2022-05-09
2022-05-10
2022-05-11
2022-05-12
2022-05-13
2022-05-14
2022-05-15
2022-05-16
2022-05-17
2022-05-18
2022-05-19
2022-05-20
2022-05-21
2022-05-22
2022-05-23
2022-05-24
2022-05-25
2022-05-26
2022-05-27
2022-05-28
2022-05-29
2022-05-30
2022-05-31
2022-06-01
2022-06-02
2022-06-03
2022-06-04
2022-06-05

'\ndf = pd.read_csv(f"TXF_1min.csv", index_col=0)\ndf.index = pd.to_datetime(df.index)\nresample_open = df[\'Open\'].resample(\'5min\').first()\nresample_high = df[\'High\'].resample(\'5min\').max()\nresample_low = df[\'Low\'].resample(\'5min\').min()\nresample_close = df[\'Close\'].resample(\'5min\').last()\nresample_volume = df[\'Volume\'].resample(\'5Min\').sum()\ndf_resample = pd.concat([resample_open, resample_high, resample_low, resample_close, resample_volume], axis=1)\ndf_resample.dropna(axis=0, inplace=True)\n# df_resample.head()\n'

In [135]:
import datetime
import pandas as pd
import os
from backtesting.lib import OHLCV_AGG

df = pd.read_csv(f"TXF_1min.csv", index_col=0)
df.index = pd.to_datetime(df.index)
df = df.resample('5T', label='right').agg(OHLCV_AGG).dropna()

In [82]:
'''
from backtesting import Strategy
from backtesting.lib import crossover
import talib as ta
from datetime import time


class KDCross(Strategy): 
    lower_bound = 25 # oversold area
    higher_bound = 75 # overbought area
    half = 50
    kd_period = 7
    div = 3 # stoch slowd_period
    n1 = 15 # fast ema
    n2 = 22 # slow ema
    macd_period = 6

    def init(self):
        self.k, self.d = self.I(ta.STOCH, high=self.data.High, low=self.data.Low, close=self.data.Close, fastk_period=self.kd_period, slowk_period=self.div, slowd_period=self.div) # line K and D
        self.macd, self.macdsignal, self.macdhist = self.I(ta.MACD, self.data.Close, self.n1, self.n2, self.macd_period) # macd(fast), macd(slow), macd histogram

    def next(self):
        # operating time : 9:15~13:30, stop loss = 0.2%, take profit = 0.4%
        if (self.data.index[-1].time() < time(13, 30)) and (self.data.index[-1].time() > time(9, 15)):
            # when k and d death cross in overbought area, close long position
            if (self.k[-1] > self.higher_bound) and (self.d[-1] > self.higher_bound) and (crossover(self.d, self.k)) and self.position.is_long:
                self.position.close()
            # when k and d golden cross in oversold area, close short position
            elif (self.k[-1] < self.lower_bound) and (self.d[-1] < self.lower_bound) and (crossover(self.k, self.d)) and self.position.is_short:
                self.position.close()
            # when k cross above d and macd cross above macdsignal and macdhist > 0, buy
            elif crossover(self.k, self.d) and (crossover(self.macd, self.macdsignal) or crossover(self.macdhist, 0)) and (self.k[-1] < self.higher_bound) and (self.d[-1] < self.higher_bound):
                if (not self.position):
                    self.buy(size=200, sl=self.data.Close[-1]*0.998, tp=self.data.Close[-1]*1.004)
                # if I have a short position, close it first
                elif (self.position.is_short) and (not self.position.is_long):
                    self.position.close()
                    self.buy(size=200, sl=self.data.Close[-1]*0.998, tp=self.data.Close[-1]*1.004)
            # when k cross below d and macd cross below macdsignal and macdhist < 0, sell
            elif crossover(self.d , self.k) and (crossover(self.macdsignal, self.macd) or crossover(0, self.macdhist)) and (self.k[-1] > self.lower_bound) and (self.d[-1] > self.lower_bound):
                if (not self.position):
                    self.sell(size=200, sl=self.data.Close[-1]*1.002, tp=self.data.Close[-1]*0.996)
                # if I have a long position, close it first
                elif (self.position.is_long) and (not self.position.is_short):
                    self.position.close()
                    self.sell(size=200, sl=self.data.Close[-1]*1.002, tp=self.data.Close[-1]*0.996)
            # when k and d are both above 50 and macd, macdsignal, and macdhist are all above 0, buy
            elif (self.k[-1] > self.d[-1]) and (self.d[-1] > self.half-15) and (self.k[-1] < self.higher_bound) and (self.d[-1] < self.higher_bound) and (self.macd > 0) and (self.macdsignal > 0) and (self.macdhist > 0):
                if (not self.position):
                    self.buy(size=200, sl=self.data.Close[-1]*0.998, tp=self.data.Close[-1]*1.004)
                # if I have a short position, close it first
                elif (self.position.is_short) and (not self.position.is_long):
                    self.position.close()
                    # self.buy(size=200, sl=self.data.Close[-1]*0.998, tp=self.data.Close[-1]*1.004)
            # when k and d are both below 50 and macd, macdsignal, and macdhist are all below  0, sell
            elif (self.k[-1] < self.d[-1]) and (self.d[-1] < self.half+15) and (self.k[-1] > self.lower_bound) and (self.d[-1] > self.lower_bound) and (self.macd < 0) and (self.macdsignal < 0) and (self.macdhist < 0):
                if (not self.position):
                    self.sell(size=200, sl=self.data.Close[-1]*1.002, tp=self.data.Close[-1]*0.996)
                # if I have a long position, close it first
                elif (self.position.is_long) and (not self.position.is_short):
                    self.position.close()
                    # self.sell(size=200, sl=self.data.Close[-1]*1.002, tp=self.data.Close[-1]*0.996)
            
            
        # close position at 13:30
        elif (self.data.index[-1].time() >= time(13, 30)) and self.position:
            self.position.close()
'''

In [167]:
from backtesting import Strategy
from backtesting.lib import crossover
import talib as ta
from datetime import time


class KDCross(Strategy): 
    lower_bound = 20 # oversold area
    higher_bound = 80 # overbought area
    half = 50
    kd_period = 9
    div = 3 # stoch slowd_period
    n1 = 19 # fast ema
    n2 = 27 # slow ema
    macd_period = 11

    def init(self):
        self.k, self.d = self.I(ta.STOCH, high=self.data.High, low=self.data.Low, close=self.data.Close, fastk_period=self.kd_period, slowk_period=self.div, slowd_period=self.div) # line K and D
        self.macd, self.macdsignal, self.macdhist = self.I(ta.MACD, self.data.Close, self.n1, self.n2, self.macd_period) # macd(fast), macd(slow), macd histogram

    def next(self):
        # operating time : 9:15~13:30, stop loss = 0.2%, take profit = 0.4%
        if (self.data.index[-1].time() < time(13, 30)) and (self.data.index[-1].time() > time(9, 15)):
            # In overbought area and any signal indicate down, close long order and sell
            if (self.k[-1] > self.higher_bound) and (self.d[-1] > self.higher_bound) and (crossover(self.d, self.k) or crossover(self.macdsignal, self.macd) or crossover(0, self.macdhist)) and self.position.is_long:
                self.position.close()
                self.sell(size=200, sl=self.data.Close[-1]*1.003, tp=self.data.Close[-1]*0.98)
            # In oversold area and any signal indicate up, close short order and buy
            elif (self.k[-1] < self.lower_bound) and (self.d[-1] < self.lower_bound) and (crossover(self.k, self.d) or crossover(self.macd, self.macdsignal) or crossover(self.macdhist, 0)) and self.position.is_short:
                self.position.close()
                self.buy(size=200, sl=self.data.Close[-1]*0.997, tp=self.data.Close[-1]*1.02)
            # k crossdown overbought, and any macd seems to be down, then close long order
            elif crossover(self.higher_bound, self.k) and (crossover(self.d, self.k) or crossover(self.macdsignal, self.macd) or crossover(0, self.macdhist)) and self.position.is_long:
                self.position.close()
            # k crossover oversold, and any macd seems to be up, then close long order
            elif crossover(self.k, self.lower_bound) and (crossover(self.k, self.d) or crossover(self.macd, self.macdsignal) or crossover(self.macdhist, 0)) and self.position.is_short:
                self.position.close()
            # In normal area, k crossover d and any macd signal indicate up, buy
            elif crossover(self.k, self.d) and (self.k[-1] < self.higher_bound) and ((self.macd > 0) or crossover(self.macd, self.macdsignal) or (self.macdhist > 0)):
                if(not self.position) and (self.k[-1] > self.half-14) and (self.d[-1] > self.half-12):
                    self.buy(size=200, sl=self.data.Close[-1]*0.997, tp=self.data.Close[-1]*1.015)
                elif (self.position.is_short) and (not self.position.is_long) and (self.k[-1] > self.half) and (self.d[-1] > self.half):
                    self.position.close()
                    self.buy(size=200, sl=self.data.Close[-1]*0.997, tp=self.data.Close[-1]*1.015)
            # In normal area, k crossdown d and any macd signal indicate down, sell
            elif crossover(self.d, self.k) and (self.k[-1] > self.lower_bound) and ((self.macd < 0) or crossover(self.macdsignal, self.macd) or (self.macdhist < 0)):
                if(not self.position) and (self.k[-1] < self.half+14) and (self.d[-1] < self.half+12):
                    self.sell(size=200, sl=self.data.Close[-1]*1.003, tp=self.data.Close[-1]*0.985)
                elif (self.position.is_long) and (not self.position.is_short) and (self.k[-1] < self.half) and (self.d[-1] < self.half):
                    self.position.close()
                    self.sell(size=200, sl=self.data.Close[-1]*1.003, tp=self.data.Close[-1]*0.985)
            # Any time, any indicator indicate other side, close position
            elif crossover(self.k, self.d) or crossover(self.macd, self.macdsignal) or crossover(self.macdhist, 0) and self.position.is_short:
                self.position.close()
            elif crossover(self.d, self.k) or crossover(self.macdsignal, self.macd) or crossover(0, self.macdhist) and self.position.is_long:
                self.position.close()
        # close position at 13:30
        elif (self.data.index[-1].time() >= time(13, 30)) and self.position:
            self.position.close()

In [168]:
from backtesting import Backtest

bt = Backtest(df, KDCross, cash=500_000, commission=0.00003, hedging=False, margin=0.05)
stats = bt.run()
print(stats.to_string())
bt.plot(resample=False)

Start                                                   2022-03-07 08:50:00
End                                                     2023-03-03 13:50:00
Duration                                                  361 days 05:00:00
Exposure Time [%]                                                 30.837212
Equity Final [$]                                                 897669.482
Equity Peak [$]                                                  906377.056
Return [%]                                                        79.533896
Buy & Hold Return [%]                                            -10.028802
Return (Ann.) [%]                                                 83.202591
Volatility (Ann.) [%]                                             41.217554
Sharpe Ratio                                                        2.01862
Sortino Ratio                                                      6.477402
Calmar Ratio                                                       6.750605
Max. Drawdow

Row(id='101312', ...)

In [160]:
df_in_sample = df[df.index >= '2022-11-06']
df_in_sample.head()
bt = Backtest(df_in_sample, KDCross, cash=500_000, commission=0.00003, hedging=False, margin=0.05)
stats = bt.optimize(kd_period=range(3, 12, 1),
                    div = range(2, 5, 1),
                    maximize='Equity Final [$]'
                    ) # 快線週期<慢線週期
print(stats.to_string())

Backtest.optimize:   0%|          | 0/9 [00:00<?, ?it/s]

Start                                                   2022-11-07 08:50:00
End                                                     2023-03-03 13:50:00
Duration                                                  116 days 05:00:00
Exposure Time [%]                                                 29.973416
Equity Final [$]                                                  572266.48
Equity Peak [$]                                                  591823.114
Return [%]                                                        14.453296
Buy & Hold Return [%]                                             18.604298
Return (Ann.) [%]                                                 56.605555
Volatility (Ann.) [%]                                              37.50496
Sharpe Ratio                                                       1.509282
Sortino Ratio                                                      4.122314
Calmar Ratio                                                       4.756709
Max. Drawdow

In [93]:
bt = Backtest(df_in_sample, KDCross, cash=500_000, commission=0.00003, hedging=False, margin=0.05)
stats = bt.optimize(n1=range(8, 16, 1),
                    n2 = range(20, 30, 1),
                    macd_period = range(6, 12, 1),
                    maximize='Equity Final [$]',
                    constraint=lambda p: p.n1 < p.n2
                    ) # 快線週期<慢線週期
print(stats.to_string())

/Users/yen/.pyenv/versions/3.7.15/lib/python3.7/site-packages/backtesting/backtesting.py:1488: UserWarning: Searching for best of 480 configurations.
  output = _optimize_grid()


Backtest.optimize:   0%|          | 0/8 [00:00<?, ?it/s]

Start                                                   2022-11-07 08:50:00
End                                                     2023-03-03 13:50:00
Duration                                                  116 days 05:00:00
Exposure Time [%]                                                 22.463447
Equity Final [$]                                                 622895.774
Equity Peak [$]                                                    630155.1
Return [%]                                                        24.579155
Buy & Hold Return [%]                                             18.604298
Return (Ann.) [%]                                                107.043579
Volatility (Ann.) [%]                                             43.264412
Sharpe Ratio                                                       2.474172
Sortino Ratio                                                      9.399197
Calmar Ratio                                                      11.041641
Max. Drawdow